In [2]:
import numpy as np
import pandas as pd
import re
import os
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Assuming flove data has already been downloaded.

In [13]:
#Reading glove word embeddings
emb_file = open("data/glove.6B.100d.txt", 'r', encoding='utf-8')
with emb_file:
    word2vec = emb_file.readlines()

In [14]:
#Creating Dictionary for the Glove word and its first 100 dimensions
temp = [word_vec.split() for word_vec in word2vec ]
word2vec_dict = {}
for words in temp:
    word2vec_dict[words[0]]=words[1:101]

In [6]:
#Read all the files from the train dictionary
files = os.listdir("data/story_summary/stories_text_summarization_dataset_train/")

Steps to perform TextRank:
    Prepare dataset of text.
    Clean, sent tokenize, word tokenize, preprocess by removing stopwords and word vectorize.
    sent vectorization
    cosine similarity matrix
    graph

In [8]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [9]:
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return re.sub(r"[^a-zA-Z0-9 .?,\"\'']", "", contractions_re.sub(replace, s))

def correct_text(passage):
    return [expand_contractions(i) for i in sent_tokenize(passage.lower())]

In [10]:
#Create a dataframe to hold the passage and its summary as highlight
df = pd.DataFrame(columns=['content','summary'])
for i,file_name in enumerate(files[:100]):
    summary, content, next = "", "", False
    with open(f'./data/story_summary/stories_text_summarization_dataset_train/{file_name}', 'r', encoding='utf-8') as file:
        for line in file.readlines():
            if len(line.replace("\n", ""))>0:
                if line.find('@highlight')>=0:
                    next=True
                    continue
                if next:
                    summary += line.replace("@highlight", "").replace("\n", "").strip().lower() + ". "
                else:
                    next=False
                    content += line.replace("\n", "").strip().lower()
        df.loc[i] = {'content':correct_text(content), 'summary':summary}

In [11]:
df.head()

,content,summary
0,[it has it is official u.s. president barack ...,syrian official: obama climbed to the top of t...
1,[cnn usain bolt rounded off the world champio...,usain bolt wins third gold of world championsh...
2,"[kansas city, missouri cnn the general servic...",the employee in agency's kansas city office is...
3,[los angeles cnn a medical doctor in vancouve...,new: a canadian doctor says she was part of a ...
4,"[cnn police arrested another teen thursday, t...",another arrest made in gang rape outside calif...


In [15]:
def passage_vec_gen(sent_list):
    sent_vec_list = []
    stop_word_list=stopwords.words('english')
    for sent in sent_list:
        vec = np.array(np.zeros(100,))
        for word in sent:
            if word not in stop_word_list and word in word2vec_dict.keys():
                vec = np.add(vec, np.asfarray(word2vec_dict[word]))
        templ = vec/len(sent)
        sent_vec_list.append(templ)
    return sent_vec_list

df['passage_sent_vec'] = [passage_vec_gen(sent_list) for sent_list in df.content]

In [17]:
def gen_cosine_sim_mat(sent_vec):
    sim_mat = np.zeros([len(sent_vec), len(sent_vec)])
    for i in range(len(sent_vec)):
        for j in range(len(sent_vec)):
            if(i != j):
                sim_mat[i][j] = cosine_similarity(sent_vec[i].reshape(1,100), sent_vec[j].reshape(1,100))[0,0]
    return(sim_mat)

df['sim_mat'] = [gen_cosine_sim_mat(sent_vec) for sent_vec in df.passage_sent_vec]

In [19]:
import tensorflow_hub as hub
import tensorflow as tf

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(df.content[0]))

In [21]:
sim_matrix = cosine_similarity(message_embeddings)
nx_graph = nx.from_numpy_array(sim_matrix)
scores = nx.pagerank(nx_graph)

In [22]:
sim_matrix.shape

(32, 32)

In [24]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(df.content[0])), reverse=True)

In [31]:
" ".join([x[1] for x in ranked_sentences[:3]])

'"the two leaders agreed that the international community must deliver a resolute message to the assad regime  and others who would consider using chemical weapons  that these crimes are unacceptable and those who violate this international norm will be held accountable by the world," the white house said.meanwhile, as uncertainty loomed over how congress would weigh in, u.s. military officials said they remained at the ready.5 key assertions u.s. intelligence report on syriasyria who wants what after chemical weapons horrorreactions mixed to obama\'s speecha spokesman for the syrian national coalition said that the opposition group was disappointed by obama\'s announcement. some u.s. lawmakers have called for immediate action while others warn of stepping into what could become a quagmire.some global leaders have expressed support, but the british parliament\'s vote against military action earlier this week was a blow to obama\'s hopes of getting strong backing from key nato allies.on

In [27]:
df.summary[0]

'syrian official: obama climbed to the top of the tree, "doesn\'t know how to get down". obama sends a letter to the heads of the house and senate. obama to seek congressional approval on military action against syria. aim is to determine whether cw were used, not by whom, says u.n. spokesman. '